In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import polars as pl
import requests
import spreg
from linearmodels.panel import PanelOLS, RandomEffects
import statsmodels.formula.api as smf
import bambi as bmb
import arviz as az 
from pysal.lib import weights
import geopandas as gpd

from shapely import wkt
import pandas as pd

import matplotlib.dates as mdates
import numpy as np
from pysal.lib import cg as geometry
import causalpy as cp
import matplotlib.pyplot as plt
dr = DataReg()

In [ ]:
df = dr.make_dataset()
df

In [ ]:
# Assuming `reg` is your DataFrame and you want to set the 'year' column as a datetime index
data = df.copy()
data["date2"] = data["year"] * 10 + data["qtr"]
data['date'] = data['date2'].astype('category')
data['zipcode'] = data['zipcode'].astype('category')
data

In [ ]:
from linearmodels.panel import RandomEffects, PanelOLS
data_panel = data.set_index(["zipcode","date2"])
data_panel = data_panel.drop("geometry", axis=1)
model = RandomEffects.from_formula(
    'total_employment ~ k_index + w_k_index + w_employment + own_children6 + own_children17 + commute_car + food_stamp + with_social_security',
    data=data_panel
)
results = model.fit(cov_type='clustered', cluster_entity=True)
print(results.summary)

In [ ]:
y = data["total_employment"].values.reshape(-1,1)
x = data[["k_index","own_children6","own_children17", "commute_car"]].values.reshape(-1,4)
w = weights.Queen.from_dataframe(df[(df["year"] == 2012) & (df["qtr"] == 1)])
fe_lag = spreg.Panel_FE_Lag(y, x, w)
print(fe_lag.summary)

In [ ]:
model = bmb.Model(
    "total_employment ~ 0 + k_index + w_k_index + w_employment + own_children6 + own_children17  + commute_car + food_stamp + with_social_security + date + zipcode",
    data, dropna=True,
)
results = model.fit(target_accept=0.99, draws=1000, cores=15)

In [ ]:
# Plot posteriors
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
res = az.summary(results)
res_filtered = res.loc[res.index.str.contains('k_index|with_social_security|w_employment')]
res_filtered

In [ ]:
res["r_hat"].max()

In [ ]:
loo = az.loo(results)
print(loo)

In [ ]:
waic = az.waic(results)
print(waic)